# Use linear regression to predict pm2.5 concentration

target: 
* OOP: a class contains hyperparameter, read file, train, predict
* Method 2: Deep Learning model
* TensorFlow: 
    * linear regression
    * gradient descent

## import library

In [1]:
#--- system ---#
import os

#--- 3rd party ---#
# data operation
import pandas as pd
import numpy as np
# ml package
import tensorflow as tf
# visualization
import matplotlib.pyplot as plt

/home/alex/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# check which jupyter it is pip or anaconda?
import sys
sys.executable

'/usr/bin/python3'

## import data

* Python API `append()` description
```python
x = [1, 2, 3]
x.append([4, 5])
print (x)
```
result: [1, 2, 3, [4, 5]]


* Python API `entend()` description
```python
x = [1, 2, 3]
x.extend([4, 5])
print (x)
```
result: [1, 2, 3, 4, 5]

In [3]:
fname = '../dataset/linear-regression-pm2.5-data/train.csv' 

df = pd.read_csv(fname, encoding='big5')
print ("Data header: {}", df.columns)

df['new_date'] = pd.to_datetime(df['日期'])
df['month'] = df['new_date'].dt.month
df['day'] = df['new_date'].dt.day

df = df.drop(columns = ['日期', '測站', 'new_date']) # only one station, dummy variable
hour_list = [str(i) for i in range(24)]
header_list = ['month', 'day', '測項']
header_list.extend(hour_list)
print (header_list)
df = df[header_list] # change column name

num_ob_term = 18 # number of m different observation terms here
df.head(num_ob_term)

Data header: {} Index(['日期', '測站', '測項', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23'],
      dtype='object')
['month', 'day', '測項', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


,month,day,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,1,1,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,1,1,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,1,1,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,1,1,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,1,1,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,1,1,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,1,1,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,1,1,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,1,1,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,1,1,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


### preprocessing
RAINFALL: 'NR' means no rain

Resource:
[Environmental Protection Administration, Executive Yuan](https://taqm.epa.gov.tw/taqm/en/HourlyData.aspx)

In [4]:
# loc choose by value in the row, a and
# change the value in the the column names which are in hour_list to 0
df.loc[df['測項'] == 'RAINFALL', hour_list] = 0 
df.head(num_ob_term)

,month,day,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,1,1,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,1,1,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,1,1,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,1,1,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,1,1,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,1,1,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,1,1,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,1,1,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,1,1,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,1,1,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


## Exploratory Data Analysis (EDA)
check whether missing value or outlier exist 

In [5]:
# check missing value, looks like no missing value(s)
df.isnull().values.any()

False

In [6]:
# check any missing observation_term, looks like no missing observation term(s)
df['測項'].value_counts()

NO2           240
WS_HR         240
NO            240
RH            240
CO            240
AMB_TEMP      240
WD_HR         240
PM10          240
CH4           240
PM2.5         240
THC           240
SO2           240
NOx           240
RAINFALL      240
WIND_DIREC    240
NMHC          240
O3            240
WIND_SPEED    240
Name: 測項, dtype: int64

## Reorganization 
* Purpose: Reorganize the dataframe and save as a new csv file for training

In [7]:
df.head(num_ob_term)

,month,day,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,1,1,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,1,1,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,1,1,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,1,1,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,1,1,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,1,1,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,1,1,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,1,1,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,1,1,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,1,1,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


In [8]:
ob_term_list = list(df['測項'].unique()) # observation terms list
print(ob_term_list)

['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC', 'WIND_SPEED', 'WS_HR']


In [9]:
# simple work around
ob_term_list.remove('PM2.5')
ob_term_list.append('PM2.5')
print(ob_term_list)

['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC', 'WIND_SPEED', 'WS_HR', 'PM2.5']


In [10]:
df_reorg = pd.DataFrame() #empty dataframe
 
for idx in range(int(len(df) / num_ob_term)): # idx range from 0 ~ 240        
    loc_tansp = df.iloc[ num_ob_term*idx:num_ob_term*(idx+1), 2: ].set_index('測項').T.copy() # get rid of dummy index at column
    loc_tansp = loc_tansp[ob_term_list] # reorder
    if idx == 0:                
        print(loc_tansp)           
    df_reorg = df_reorg.append(loc_tansp)

測項 AMB_TEMP  CH4    CO  NMHC   NO  NO2  NOx  O3 PM10 RAINFALL  RH  SO2  THC  \
0        14  1.8  0.51   0.2  0.9   16   17  16   56        0  77  1.8    2   
1        14  1.8  0.41  0.15  0.6  9.2  9.8  30   50        0  68    2    2   
2        14  1.8  0.39  0.13  0.5  8.2  8.7  27   48        0  67  1.7    2   
3        13  1.8  0.37  0.12  1.7  6.9  8.6  23   35        0  74  1.6  1.9   
4        12  1.8  0.35  0.11  1.8  6.8  8.5  24   25        0  72  1.9  1.9   
5        12  1.8   0.3  0.06  1.5  3.8  5.3  28   12        0  73  1.4  1.8   
6        12  1.8  0.37   0.1  1.9  6.9  8.8  24    4        0  74  1.5  1.9   
7        12  1.8  0.47  0.13  2.2  7.8  9.9  22    2        0  73  1.6  1.9   
8        15  1.8  0.78  0.26  6.6   15   22  21   11        0  66  5.1  2.1   
9        17  1.8  0.74  0.23  7.9   21   29  29   38        0  56   15    2   
10       20  1.8  0.59   0.2  4.2   14   18  44   56        0  45  4.5    2   
11       22  1.8  0.52  0.18  2.9   11   14  58   64

In [11]:
df_reorg.head(24)

測項,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR,PM2.5
0,14,1.8,0.51,0.2,0.9,16,17,16,56,0,77,1.8,2,37,35,1.4,0.5,26
1,14,1.8,0.41,0.15,0.6,9.2,9.8,30,50,0,68,2,2,80,79,1.8,0.9,39
2,14,1.8,0.39,0.13,0.5,8.2,8.7,27,48,0,67,1.7,2,57,2.4,1,0.6,36
3,13,1.8,0.37,0.12,1.7,6.9,8.6,23,35,0,74,1.6,1.9,76,55,0.6,0.3,35
4,12,1.8,0.35,0.11,1.8,6.8,8.5,24,25,0,72,1.9,1.9,110,94,1.7,0.6,31
5,12,1.8,0.3,0.06,1.5,3.8,5.3,28,12,0,73,1.4,1.8,106,116,2.5,1.9,28
6,12,1.8,0.37,0.1,1.9,6.9,8.8,24,4,0,74,1.5,1.9,101,106,2.5,2,25
7,12,1.8,0.47,0.13,2.2,7.8,9.9,22,2,0,73,1.6,1.9,104,94,2,2,20
8,15,1.8,0.78,0.26,6.6,15,22,21,11,0,66,5.1,2.1,124,232,0.6,0.5,19
9,17,1.8,0.74,0.23,7.9,21,29,29,38,0,56,15,2,46,153,0.8,0.3,30


In [12]:
# set the name of columns' to None, get rid of '測項' of dataframe column name
df_reorg.columns.name = None
# keep header, insead of index
df_reorg.to_csv('../dataset/linear-regression-pm2.5-data/train_reorganized.csv', header=True, index=False)

## Step 1 Define you function set (Model)
* X is the observation value of each terms
* y is the concentration of PM$_{2.5}$

Math equation should be
<br>y = b + w$_{1,pm10}$ * x$_{1,pm10}$ + ... + w$_{9,pm10}$ * x$_{9,pm10}$
<br>&emsp; + w$_{1,NO}$ * x$_{1,NO}$ + ... + w$_{9,NO}$ * x$_{9,NO}$
<br>&emsp; ...
<br>&emsp; + w$_{1, O_3}$ * x$_{1,O_3}$ + ... + w$_{9,O_3}$ * x$_{9,O_3}$ 


In [13]:
fname = '../dataset/linear-regression-pm2.5-data/train_reorganized.csv' 
df = pd.read_csv(fname)
df.head(5)

,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR,PM2.5
0,14.0,1.8,0.51,0.20,0.9,16.0,17.0,16.0,56,0,77,1.8,2.0,37.0,35.0,1.4,0.5,26
1,14.0,1.8,0.41,0.15,0.6,9.2,9.8,30.0,50,0,68,2.0,2.0,80.0,79.0,1.8,0.9,39
2,14.0,1.8,0.39,0.13,0.5,8.2,8.7,27.0,48,0,67,1.7,2.0,57.0,2.4,1.0,0.6,36
3,13.0,1.8,0.37,0.12,1.7,6.9,8.6,23.0,35,0,74,1.6,1.9,76.0,55.0,0.6,0.3,35
4,12.0,1.8,0.35,0.11,1.8,6.8,8.5,24.0,25,0,72,1.9,1.9,110.0,94.0,1.7,0.6,31


### Seperate training set and validation set
`mod = a % b`

In [14]:
sep_day_list = [x for x in range(16,21)]
print(sep_day_list)

[16, 17, 18, 19, 20]


In [15]:
month_window_size = 20 # only 20 days per month in this dataset
num_hours_a_day = 24

# create empty dataframe
train_df = pd.DataFrame()
validation_df = pd.DataFrame()

for idx in range(int(len(df) / num_hours_a_day)): # total: 240 days    
    if idx % month_window_size not in sep_day_list:
        train_df = train_df.append(df.iloc[idx * num_hours_a_day: (idx+1) * num_hours_a_day]) # get first 16 days of each month
    else:
        validation_df = validation_df.append(df.iloc[idx * num_hours_a_day: (idx+1) * num_hours_a_day]) #

### Examine the equity of two float number...
* [What is the best way to compare floats for almost-equality in Python?](https://stackoverflow.com/questions/5595425/what-is-the-best-way-to-compare-floats-for-almost-equality-in-python)
* [numpy.isclose](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.isclose.html)

In [16]:
# check data size
train_days = int(len(train_df) / num_hours_a_day)
validation_days = int(len(validation_df) / num_hours_a_day)

print('training dataset: {} days, validation dataset: {} days'.format(train_days, validation_days))

train_ratio = (8.0 / 10.0)
validation_ratio = 1 - train_ratio # note: int - float = int
print('train_ratio = {:.2f}, validation_ratio = {:.2f}'.format(train_ratio, validation_ratio))

print ('train_days ratio = {:.2f}, validation_days ratio = {:.2f}'\
       .format( float(train_days / (train_days + validation_days)), \
                float(validation_days / (train_days + validation_days))))

training dataset: 192 days, validation dataset: 48 days
train_ratio = 0.80, validation_ratio = 0.20
train_days ratio = 0.80, validation_days ratio = 0.20


In [17]:
# float number close enough?
assert(np.isclose(float(train_days / (train_days + validation_days)), train_ratio, rtol=1e-05, atol=1e-08, equal_nan=False) == True) 

In [18]:
# float number close enough?
np.isclose(float(validation_days / (train_days + validation_days)), validation_ratio, rtol=1e-05, atol=1e-08, equal_nan=False)

True

* train_x: each data point contrains (N-9) ~ (N-1) hours of 
* train_y: the concentration of Nth hour PM$_{2.5}$

In [38]:
def data_preparation(input_dataframe, num_prev_hours):
    train_x = pd.DataFrame()    
    
    # train_x
    if int(len(input_dataframe)) > 0:
        for idx in range(int(len(input_dataframe))):  # total data in training dataset: 16 days * 12(months) * 24(hours)
            try:
                print (input_dataframe.iloc[idx, :])
                for hour in num_prev_hours:
                    train_x = pd.concat([input_dataframe.iloc[idx + hour, :], train_x])                                                                   
                    print(train_x)                
            except: # out of table bound
                break;    
    # train_y 
    if int(len(input_dataframe['PM2.5'])) > 0:
        train_y = input_dataframe['PM2.5'].iloc[num_prev_hours:]
    
    return train_x, train_y 

In [39]:
train_x = pd.DataFrame()
train_y = pd.DataFrame()

train_x, train_y  = data_preparation(df, 9)

AMB_TEMP      14.00
CH4            1.80
CO             0.51
NMHC           0.20
NO             0.90
NO2           16.00
NOx           17.00
O3            16.00
PM10          56.00
RAINFALL       0.00
RH            77.00
SO2            1.80
THC            2.00
WD_HR         37.00
WIND_DIREC    35.00
WIND_SPEED     1.40
WS_HR          0.50
PM2.5         26.00
Name: 0, dtype: float64


In [40]:
print (train_x)

Empty DataFrame
Columns: []
Index: []


In [43]:
print (train_y[:5])

9     30
10    41
11    44
12    33
13    37
Name: PM2.5, dtype: int64


In [ ]:
X = tf.placeholder(tf.float32) # vector length= 3
Y = tf.placeholder(tf.float32)

num_hours = 9

W = tf.Variable(np.random.randn(), name='weights') # add one b into w matrix
B = tf.Variable(np.random.randn(), name='bias')

In [ ]:
# hyperparamters
learning_rate = 0.01
epochs = 1000

In [ ]:
pred = W * X + B # two matrix multiplication, tf.matmul(X, w) 
cost = tf.reduce_sum((pred - Y) ** 2) / (2 * len(train_x)) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
%matplotlib inline

In [ ]:
# with tf.Session() as sess:
#     sess.run(init)
#     for x, y in zip(train_x, train_y):
#         sess.run(optimizer, feed_dict={X: x, Y: y})
# 
#     if not epoch % 10:
#         loss = sess.run(cost, feed_dict={X: train_x, Y: train_y})    
#     plt.plot(train_x, train_y, 'o')
#     plt.plot(train_x, a_epoch * train_x**3 + b_epoch * train_x**2 + c_epoch * train_x + d_epoch)
#     plt.show()